## Урок 4. Системы линейных уравнений

### Системы линейных уравнений. Часть 1


__1.__ Решить систему уравнений методом Гаусса:

$$\begin{cases}
x_{1}+x_{2}-x_{3}-2x_{4}=0, \\
2x_{1}+x_{2}-x_{3}+x_{4}=-2, \\
x_{1}+x_{2}-3x_{3}+x_{4}=4.
\end{cases}$$


__Решение__

Данная система уравнений является _недоопределенной_, так как число уравнений 3 меньше числа неизвестных 4.

Запишем расширенную матрицу системы:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 1 & -1 & -2\\ 
2 & 1 & -1 & 1\\ 
1 & 1 & -3 & 1
\end{matrix}\right|
\begin{matrix}
0\\ 
-2\\
4
\end{matrix}
\end{pmatrix}.$$

Путем элементарных преобразований постараемся привести матрицу к ступенчатому виду.

Вычтем из второй строки первую, умноженную на 2:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 1 & -1 & -2\\ 
0 & -1 & 1 & 5\\ 
1 & 1 & -3 & 1
\end{matrix}\right|
\begin{matrix}
0\\ 
-2\\
4
\end{matrix}
\end{pmatrix}.$$

Вычтем из третьей строки первую:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 1 & -1 & -2\\ 
0 & -1 & 1 & 5\\ 
0 & 0 & -2 & 3
\end{matrix}\right|
\begin{matrix}
0\\ 
-2\\
4
\end{matrix}
\end{pmatrix}.$$


В полученном ступенчатом виде мы имеем ступеньки из двух чисел, а не из одного, как это требуется. В таком случае система имеет _бесконечное количество решений._

Оставшаяся матрица соответствует системе

$$\left\{\begin{array}{r}
x_{1}+x_{2}-x_{3}-2x_{4}=0, \\
-x_{2}+x_{3}+5x_{4}=-2, \\
-2x_{3}+3x_{4}=4.
\end{array}\right.$$

Бесконечное множество решений необходимо записывать в виде _общего решения_.

Последнее уравнение содержит две неизвестных. Неизвестные, «сидящие» на ступеньках матрицы, являются _базисными_, и их нужно выразить через _свободные_ неизвестные. В данном случае можно рассматривать в качестве _свободного_ параметра $x_{4}=c$. Тогда, выражая остальные переменные через $c$, получим:

$x_{4}=c,$

$-2x_{3}+3c=4 \; \Rightarrow \; x_{3}=\frac{3c-4}{2}=\frac{3}{2}c-2,$

$-x_{2} + \frac{3}{2}c-2 + 5c=-2 \; \Rightarrow \; x_{2}=\frac{3}{2}c-2 + 5c + 2=\frac{13}{2}c,$

$x_{1} + \frac{13}{2}c - \left(\frac{3}{2}c-2\right) - 2c=0 \; \Rightarrow \; x_{1}= -\frac{13}{2}c + \frac{3}{2}c - 2 + 2c=-\frac{6}{2}c - 2=-3c-2.$

Таковым будет _общее решение системы_. Подставляя произвольные числа вместо $c$, мы получим _частное решение_. Например, при $c=0$:

$x_{1}=-2,$

$x_{2}=0,$

$x_{3}=-2,$

$x_{4}=0.$

Параметр $c$ может принимать бесконечное количество значений, при которых уравнения в системе будут обращаться в тождества.

Подставив общее решение в исходную систему, получим тождества:

$$\begin{cases}
-3c-2+\frac{13}{2}c-(\frac{3}{2}c-2)-2c=0, \\
2\cdot(-3c-2)+\frac{13}{2}c-(\frac{3}{2}c-2)+c=-2, \\
-3c-2+\frac{13}{2}c-3\cdot(\frac{3}{2}c-2)+c=4.
\end{cases}$$

Подставив частное решение в исходную систему, также получим тождества:

$$\begin{cases}
-2+0+2-2\cdot0=0, \\
2\cdot(-2)+0+2+0=-2, \\
-2+0-3\cdot(-2)+0=4.
\end{cases}$$


__2.__ Проверить на совместность и выяснить, сколько решений будет иметь система линейных уравнений:

   а) $\begin{cases}
3x_{1}-x_{2}+x_{3}=4, \\
2x_{1}-5x_{2}-3x_{3}=-17, \\
x_{1}+x_{2}-x_{3}=0;
\end{cases}$


__Решение__

Векторы коэффициентов линейно независимы, следовательно система _совместная и определенная, т.е. имеет единственное решение_.

Решим систему методом Гаусса:

$$\begin{pmatrix}
\left.\begin{matrix}
3 & -1 & 1 \\
2 & -5 & -3 \\ 
1 & 1 & -1
\end{matrix}\right|
\begin{matrix}
4 \\
-17 \\ 
0
\end{matrix}
\end{pmatrix},$$

переставим строки в порядке возрастания первого элемента строки $a_{i1}$:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 1 & -1 \\
2 & -5 & -3 \\ 
3 & -1 & 1
\end{matrix}\right|
\begin{matrix}
0 \\
-17 \\ 
4
\end{matrix}
\end{pmatrix},$$

вычтем из второй и третьей строки первую строку, умноженную на 2 и 3 соответственно:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 1 & -1 \\
0 & -7 & -1 \\ 
0 & -4 & 4
\end{matrix}\right|
\begin{matrix}
0 \\
-17 \\ 
4
\end{matrix}
\end{pmatrix},$$

вычтем из третьей строки, умноженной предварительно на 7, вторую строку, умноженную на 4:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 1 & -1 \\
0 & -7 & -1 \\
0 & 0 & 32
\end{matrix}\right|
\begin{matrix}
0 \\
-17 \\ 
96
\end{matrix}
\end{pmatrix}.$$


В итоге после преобразований мы получили матрицу треугольного вида, которая представляет следующую систему:

$$\left\{\begin{array}{r}
x_{1}+x_{2}-x_{3}=0, \\
-7x_{2}-x_{3}=-17, \\
32x_{3}=96.
\end{array}\right.$$

Проводя обратный ход метода Гаусса, последовательно подставляя известные значения в уравнения, получим

$$x_{3}=\frac{96}{32}=3.$$

Подставим $x_{3}$ во второе уравнение:
$$-7x_{2}-3=-17 \Rightarrow x_{2}=\frac{17-3}{7}=2,$$

а затем полученные $x_{2}$ и $x_{3}$ — в первое уравнение:
$$x_{1}+2-3=0 \Rightarrow x_{1}=3-2=1.$$

Если подставить полученные значения неизвестных в исходную систему, несложно убедиться, что все выражения обращаются в тождества:

$\begin{cases}
3\cdot1-2+3=4, \\
2\cdot1-5\cdot2-3\cdot3=-17, \\
1+2-3=0.
\end{cases}$

    
   б) $\begin{cases}
2x_{1}-4x_{2}+6x_{3}=1, \\
x_{1}-2x_{2}+3x_{3}=-2, \\
3x_{1}-6x_{2}+9x_{3}=5;
\end{cases}$


__Решение__

Векторы коэффициентов линейно зависимы, и вектор свободных членов не принадлежит одномерному подпространству, которое они образуют, следовательно система _несовместная, т.е. не имеет решений_.

Решим систему методом Гаусса:

$$\begin{pmatrix}
\left.\begin{matrix}
2 & -4 & 6 \\
1 & -2 & 3 \\ 
3 & -6 & 9
\end{matrix}\right|
\begin{matrix}
1 \\
-2 \\ 
5
\end{matrix}
\end{pmatrix},$$

вычтем из первой и третьей строки вторую строку, умноженную на 2 и 3 соответственно:

$$\begin{pmatrix}
\left.\begin{matrix}
0 & 0 & 0 \\
1 & -2 & 3 \\ 
0 & 0 & 0
\end{matrix}\right|
\begin{matrix}
5 \\
-2 \\ 
11
\end{matrix}
\end{pmatrix}.$$

Видно, что система _несовместная_, так как полученные равенства в первой и третьей строке $0 = 5$ и $0 = 11$ не имеют смысла.

    
   в) $\begin{cases}
x_{1}+2x_{2}+5x_{3}=4, \\
3x_{1}+x_{2}-8x_{3}=-2. 
\end{cases}$


__Решение__

Данная система уравнений является _совместной и недоопределенной_, так как число уравнений 2 меньше числа неизвестных 3.

В двумерном пространстве любая пара линейно независимых векторов, как в данном случае

$$a_{1}=\begin{pmatrix}
1\\ 
3
\end{pmatrix}, a_{2}=
\begin{pmatrix}
2\\ 
1
\end{pmatrix}, a_{3}=
\begin{pmatrix}
5\\ 
-8
\end{pmatrix}$$

может образовать базис, по которому получится единственным образом разложить любой вектор. В частности, перенося $a_{3}x_{3}$ в правую часть уравнения вида $a_{1}x_{1}+a_{2}x_{2}+a_{3}x_{3}=b$, мы сможем однозначно разложить получившийся справа вектор $b-a_{3}x_{3}$ и получить тождество, но, так как $x_{3}$ может принимать бесконечное количество значений, то и _решений системы также будет бесконечное количество_.

Решим систему методом Гаусса:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 2 & 5\\ 
3 & 1 & -8
\end{matrix}\right|
\begin{matrix}
4\\ 
-2
\end{matrix}
\end{pmatrix}.$$

Вычтем из второй строки первую, умноженную на 3:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 2 & 5\\ 
0 & -5 & -23
\end{matrix}\right|
\begin{matrix}
4\\ 
-14
\end{matrix}
\end{pmatrix}.$$

В полученном ступенчатом виде мы имеем ступеньки из двух чисел, а не из одного, как это требуется. В таком случае система имеет _бесконечное количество решений._

Оставшаяся матрица соответствует системе

$$\left\{\begin{array}{r}
x_{1}+2x_{2}+5x_{3}=4, \\
-5x_{2}-23x_{3}=-14. 
\end{array}\right.$$

Бесконечное множество решений необходимо записывать в виде _общего решения_.


Последнее уравнение содержит две неизвестных. Неизвестные, «сидящие» на ступеньках матрицы, являются _базисными_, и их нужно выразить через _свободные_ неизвестные. В данном случае можно рассматривать в качестве _свободного_ параметра $x_{3}=c$. Тогда, выражая остальные переменные через $c$, получим:

$x_{3}=c,$

$-5x_{2}-23с=-14 \; \Rightarrow \; x_{2}=\frac{14-23c}{5},$

$x_{1}+2\cdot\frac{14-23c}{5}+5с=4 \; \Rightarrow \; x_{1}=\frac{46c-28}{5}-5с+4=\frac{21c-28}{5}+4.$

Таковым будет _общее решение системы_. Подставляя произвольные числа вместо $c$, мы получим _частное решение_. Например, при $c=0$:

$x_{1}=-\frac{8}{5},$

$x_{2}=\frac{14}{5},$

$x_{3}=0.$

Параметр $c$ может принимать бесконечное количество значений, при которых уравнения в системе будут обращаться в тождества.

Подставив общее решение в исходную систему, получим тождества:

$$\begin{cases}
\frac{21c-28}{5}+4+2\cdot\frac{14-23c}{5}+5c=4, \\
3\cdot\left(\frac{21c-28}{5}+4\right)+\frac{14-23c}{5}-8c=-2. 
\end{cases}$$

Подставив частное решение в исходную систему, также получим тождества:

$$\begin{cases}
-\frac{8}{5}+2\cdot\frac{14}{5}+5\cdot0=4, \\
-3\cdot\frac{8}{5}+\frac{14}{5}-8\cdot0=-2. 
\end{cases}$$


__3.__ Проверить на совместность и выяснить, сколько решений будет иметь система линейных уравнений, заданная расширенной матрицей

$$\tilde{A}=\begin{pmatrix}
\left.\begin{matrix}
1 & 3 & -2 & 4\\ 
0 & 5 & 0 & 1\\ 
0 & 0 & 3 & 0\\ 
0 & 0 & 0 & 2
\end{matrix}\right|
\begin{matrix}
3\\ 
2\\
4\\
1
\end{matrix}
\end{pmatrix}.$$

__Решение__

В основной и расширенной матрице есть минор 4-го порядка с ненулевым определителем, так как главная диагональ без нулей, поэтому $rank A=rank \tilde A=n=4$. В этом случае система _совместная и определенная, т.е. имеет единственное решение_.

In [31]:
import numpy as np
A = np.array([[1, 3, -2, 4],
              [0, 5, 0, 1],
              [0, 0, 3, 0],
              [0, 0, 0, 2]])
A_ = np.array([[1, 3, -2, 4, 3],
              [0, 5, 0, 1, 2],
              [0, 0, 3, 0, 4],
              [0, 0, 0, 2, 1]])

np.linalg.matrix_rank(A), np.linalg.matrix_rank(A_)

(4, 4)

Данная матрица треугольного вида представляет следующую систему:

$$\left\{\begin{array}{r}
x_{1}+3x_{2}-2x_{3}+4x_{4}=3, \\
5x_{2}+x_{4}=2, \\
3x_{3}=4, \\
2x_{4}=1.
\end{array}\right.$$

Проводя обратный ход метода Гаусса, последовательно подставляя известные значения в уравнения, получим

$$x_{4}=\frac{1}{2},
x_{3}=\frac{4}{3}.$$

Подставим $x_{4}$ во второе уравнение:
$$5x_{2}+\frac{1}{2}=2 \Rightarrow x_{2}=\frac{3}{10},$$

а затем полученные $x_{2}$, $x_{3}$ и $x_{4}$ — в первое уравнение:
$$x_{1}+3\cdot\frac{3}{10}-2\cdot\frac{4}{3}+4\cdot\frac{1}{2}=3 \Rightarrow x_{1}=3-\frac{9}{10}+\frac{8}{3}-2=1+\frac{80-27}{30}=\frac{83}{30}.$$

Если подставить полученные значения неизвестных в исходную систему, несложно убедиться, что все выражения обращаются в тождества:

$\begin{cases}
\frac{83}{30}+3\cdot\frac{3}{10}-2\cdot\frac{4}{3}+4\cdot\frac{1}{2}=3, \\
5\cdot\frac{3}{10}+\frac{1}{2}=2, \\
3\cdot\frac{4}{3}=4, \\
2\cdot\frac{1}{2}=1.
\end{cases}$


__4.__ Дана система линейных уравнений, заданная расширенной матрицей

$$\tilde{A}=\begin{pmatrix}
\left.\begin{matrix}
1 & 2 & 3\\ 
4 & 5 & 6\\ 
7 & 8 & 9
\end{matrix}\right|
\begin{matrix}
a\\ 
b\\
c
\end{matrix}
\end{pmatrix}.$$

Найти соотношение между параметрами $a$, $b$ и $c$, при которых система является несовместной.


__Решение__

Приведем матрицу к треугольному виду с помощью метода Гаусса:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 2 & 3 \\
4 & 5 & 6 \\ 
7 & 8 & 9
\end{matrix}\right|
\begin{matrix}
a \\
b \\ 
c
\end{matrix}
\end{pmatrix},$$

вычтем из второй и третьей строки первую строку, умноженную на 4 и 7 соответственно:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 2 & 3 \\
0 & -3 & -6 \\ 
0 & -6 & -12
\end{matrix}\right|
\begin{matrix}
a \\
b-4a \\ 
c-7a
\end{matrix}
\end{pmatrix},$$

вычтем из третьей строки вторую строку, умноженную на 2:

$$\begin{pmatrix}
\left.\begin{matrix}
1 & 2 & 3 \\
0 & -3 & -6 \\ 
0 & 0 & 0
\end{matrix}\right|
\begin{matrix}
a \\
b-4a \\ 
c+a-2b
\end{matrix}
\end{pmatrix}.$$


Так как получен 0 в главной диагонали, то в основной  матрице есть максимум минор 2-го порядка с ненулевым определителем, 
а в расширенной матрице кроме минора 2-го порядка с ненулевым определителем, может быть и минор 3-го порядка с ненулевым определителем в случае если выражение $c+a-2b$ не будет равно нулю.

Таким образом, если выражение $c+a-2b$ не будет равно нулю, то $rank \tilde A=3$, в то время как $rank A=2$. Это означает, что система в этом случае _несовместная_.

In [70]:
# Например, один или два из параметров a, b и c равны нулю, а остальные равны 1
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
a = 0
b = 1
c = 1
A_ = np.array([[1, 2, 3, a],
              [4, 5, 6, b],
              [7, 8, 9, c]])

np.linalg.matrix_rank(A), np.linalg.matrix_rank(A_)

(2, 3)

В остальных случаях если выражение $c+a-2b$ будет равно нулю, тогда $rank A=rank \tilde A=2$, что меньше количества неизвестных $n=3$. В этом случае система имеет бесконечное множество решений, и одна из неизвестных должна рассматриваться как свободный параметр, при любых значениях которого система будет иметь решение.

In [71]:
# Например, все параметры a, b и c равны нулю или все равны 1 (тогда c+a-2b = 1+1-2 = 0)
a = 0
b = 0
c = 0
A_ = np.array([[1, 2, 3, a],
              [4, 5, 6, b],
              [7, 8, 9, c]])

np.linalg.matrix_rank(A), np.linalg.matrix_rank(A_)

(2, 2)

In [72]:
# Например, b=3, а a=5 и c=1 (или наоборот a=1 и c=5, тогда c+a-2b = 1+5-2*3 = 0)
a = 5
b = 3
c = 1
A_ = np.array([[1, 2, 3, a],
              [4, 5, 6, b],
              [7, 8, 9, c]])

np.linalg.matrix_rank(A), np.linalg.matrix_rank(A_)

(2, 2)

### Системы линейных уравнений. Часть 2

__1.__ Решить систему уравнений методом Крамера:

   а) $\begin{cases}
x_{1}-2x_{2}=1 \\
3x_{1}-4x_{2}=7
\end{cases}$

    
   б) $\begin{cases}
2x_{1}-x_{2}+5x_{3}=10 \\
x_{1}+x_{2}-3x_{3}=-2 \\
2x_{1}+4x_{2}+x_{3}=1
\end{cases}$

__2.__ Найти $L$-матрицу $LU$-разложения для матрицы коэффициентов:

   а)$$\begin{pmatrix}
1 & 2 & 4 \\ 
2 & 9 & 12 \\ 
3 & 26 & 30
\end{pmatrix}$$

    
   б)$$\begin{pmatrix}
1 & 1 & 2 & 4\\ 
2 & 5 & 8 & 9\\ 
3 & 18 & 29 & 18\\
4 & 22 & 53 & 33
\end{pmatrix}$$

    
__3.__ Решить систему линейных уравнений методом $LU$-разложения

$$\begin{cases}
2x_{1}+x_{2}+3x_{3}=1 \\
11x_{1}+7x_{2}+5x_{3}=-6 \\
9x_{1}+8x_{2}+4x_{3}=-5
\end{cases}$$

__4.__ Решить систему линейных уравнений методом Холецкого

$$\begin{cases}
81x_{1}-45x_{2}+45x_{3}=531 \\
-45x_{1}+50x_{2}-15x_{3}=-460 \\
45x_{1}-15x_{2}+38x_{3}=193
\end{cases}$$


__5*.__ Написать на Python программу с реализацией одного из изученных алгоритмов решения СЛАУ.